# Find all PDBs associated with UniProt AC.

Conny Yu, 08/2024<br><br>
Using UniProt API, find all PDBs associated to the UniProt entry.<br>
Using PDB API, find all references (including preprints) and their PMIDs associated to each PDB.<br>
Input: UniProt AC.

In [5]:
import requests
from collections import defaultdict

def get_pdb_codes(uniprot_ac):
    # Query UniProt API to get the PDB information
    uniprot_url = f"https://www.uniprot.org/uniprot/{uniprot_ac}.xml"
    response = requests.get(uniprot_url)
    
    if response.status_code != 200:
        print(f"\033[1mError fetching data from UniProt for AC {uniprot_ac}\033[0m")
        return []
    
    xml_data = response.text
    
    # Extract the PDB codes from the XML data
    pdb_codes = set()
    for line in xml_data.splitlines():
        if 'dbReference type="PDB"' in line:
            start = line.find('id="') + 4
            end = line.find('"', start)
            pdb_code = line[start:end]
            pdb_codes.add(pdb_code)
    
    return list(pdb_codes)

def get_pdb_pmids(pdb_code):
    # Query RCSB PDB API to get references for the PDB code
    pdb_url = f"https://data.rcsb.org/rest/v1/core/entry/{pdb_code}"
    response = requests.get(pdb_url)
    
    if response.status_code != 200:
        print(f"\033[1mError fetching data from RCSB PDB for PDB code {pdb_code}\033[0m")
        return []
    
    pdb_data = response.json()
    
    # Extract PMIDs from the PDB data
    pmids = set()
    if 'rcsb_primary_citation' in pdb_data:
        primary_citation = pdb_data['rcsb_primary_citation']
        if 'pdbx_database_id_pub_med' in primary_citation:
            pmids.add(primary_citation['pdbx_database_id_pub_med'])
    
    if 'citation' in pdb_data:
        for citation in pdb_data['citation']:
            if 'pdbx_database_id_pub_med' in citation:
                pmids.add(citation['pdbx_database_id_pub_med'])
    
    return list(pmids)

def main():
    # Prompt the user for the UniProt AC number
    uniprot_ac = input("Enter UniProt AC: ").strip()
    
    # Get the list of PDB codes associated with the given UniProt AC number
    pdb_codes = get_pdb_codes(uniprot_ac)
    
    pmid_to_pdb = defaultdict(list)
    pdb_to_pmid = {}
    all_pmids = set()
    
    if pdb_codes:
        for pdb_code in pdb_codes:
            pmids = get_pdb_pmids(pdb_code)
            for pmid in pmids:
                pmid_to_pdb[pmid].append(pdb_code)
                pdb_to_pmid[pdb_code] = pmid
                all_pmids.add(pmid)
        
        print(f"\033[1mPMIDs and associated PDB codes for UniProt AC {uniprot_ac}:\033[0m")
        
        # Sort PMIDs in descending order
        for pmid in sorted(pmid_to_pdb.keys(), reverse=True):
            print(f"PMID: {pmid}")
            print(f"  Associated PDBs: {', '.join(pmid_to_pdb[pmid])}")

        # Sort PDBs by descending order of their associated PMIDs
        sorted_pdbs = sorted(pdb_codes, key=lambda pdb: pdb_to_pmid.get(pdb, 0), reverse=True)
        
        # All associated PDBs
        print("\n\033[1mAll associated PDBs:\033[0m")
        print(", ".join(str(pdb_code).lower() for pdb_code in sorted_pdbs))
        
        # Output for Asterix GM
        print("\n\033[1mAll associated references:\033[0m")
        print(", ".join(str(pmid) for pmid in sorted(all_pmids, reverse=True)))
        
        # Output for PyMOL command
        print("\n\033[1mTo use in Pymol command:\033[0m")
        print("fetch " + "; fetch ".join(str(pdb_code).lower() for pdb_code in sorted_pdbs))
    else:
        print(f"No PDB codes found for UniProt AC {uniprot_ac}.")

if __name__ == "__main__":
    main()


Enter UniProt AC:  Q9Y2U9


PMIDs and associated PDB codes for UniProt AC Q9Y2U9:
PMID: 38177675
  Associated PDBs: 8SH2, 8SGF, 8SGE
PMID: 36805027
  Associated PDBs: 8EBL, 8EBN, 8EBM
PMID: 30526872
  Associated PDBs: 6DO3, 6DO5, 6DO4

All associated PDBs:
8sh2, 8sgf, 8sge, 8ebl, 8ebn, 8ebm, 6do3, 6do5, 6do4, 8pif

All associated references:
38177675, 36805027, 30526872

To use in Pymol command:
fetch 8sh2; fetch 8sgf; fetch 8sge; fetch 8ebl; fetch 8ebn; fetch 8ebm; fetch 6do3; fetch 6do5; fetch 6do4; fetch 8pif
